In [90]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression

from scipy.stats import uniform, randint, loguniform
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
sns.set_style("whitegrid", {'grid.linestyle': '--'})
sns.set_style({'font.family':'serif', 'font.serif':'Computer Modern'})
sns.set_context(font_scale=2, rc={"font.size":10,"axes.titlesize":20,"axes.labelsize":15})

In [3]:
df_train = pd.read_csv("df_train_processed.csv")
df_test = pd.read_csv("df_test_processed.csv")

In [4]:
df_train.head()

,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,frame_count,sum,mean,...,stft_std_w4,stft_min_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_kur_w4,stft_skew_w4
0,speech,neutral,normal,Kids are talking by the door,1st,1,M,158558,0.145081,9.150000e-07,...,0.155455,0.000000,0.000000,0.566462,0.709962,0.799141,0.896606,1.0,6.742190,-1.706215
1,speech,neutral,normal,Kids are talking by the door,2nd,1,M,160160,0.114319,7.137790e-07,...,0.163183,0.270133,0.368623,0.472736,0.623183,0.744908,0.874713,1.0,-0.700420,-0.201495
2,speech,neutral,normal,Dogs are sitting by the door,1st,1,M,156956,0.149963,9.554485e-07,...,0.190634,0.000000,0.000000,0.417919,0.643636,0.774253,0.899156,1.0,1.688986,-1.024773
3,speech,neutral,normal,Dogs are sitting by the door,2nd,1,M,152152,0.139618,9.176213e-07,...,0.182551,0.205616,0.306280,0.399641,0.606910,0.755213,0.886474,1.0,-0.594111,-0.412871
4,speech,calm,normal,Kids are talking by the door,1st,1,M,169769,0.137665,8.108948e-07,...,0.177069,0.177847,0.248765,0.428202,0.634815,0.759914,0.878014,1.0,0.126535,-0.620782


In [5]:
df_test.head()

,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,frame_count,sum,mean,...,stft_std_w4,stft_min_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_kur_w4,stft_skew_w4
0,speech,neutral,normal,Kids are talking by the door,1st,19,M,161762,0.475616,2.940224e-06,...,0.169929,0.222808,0.306697,0.466157,0.645931,0.789123,0.908316,1.0,-0.260279,-0.566556
1,speech,neutral,normal,Kids are talking by the door,2nd,19,M,155355,0.120148,7.733752e-07,...,0.216928,0.114567,0.170786,0.279347,0.557524,0.733053,0.866973,1.0,-0.428900,-0.587210
2,speech,neutral,normal,Dogs are sitting by the door,1st,19,M,160160,-0.599304,-3.741909e-06,...,0.227178,0.010295,0.023285,0.305395,0.577750,0.741576,0.889984,1.0,0.626916,-0.883317
3,speech,neutral,normal,Dogs are sitting by the door,2nd,19,M,158558,-0.080383,-5.069646e-07,...,0.179467,0.241405,0.368081,0.434591,0.620144,0.776419,0.904668,1.0,-0.825144,-0.383701
4,speech,calm,normal,Kids are talking by the door,1st,19,M,176176,0.152222,8.640319e-07,...,0.189124,0.217860,0.298851,0.393112,0.623309,0.772964,0.907035,1.0,-0.536669,-0.547354


## Standardization

### Train

In [6]:
df_train_num = df_train.select_dtypes(include="number").drop('actor', axis=1)
df_train_cat = df_train.select_dtypes(include=['object'])

scaler = MinMaxScaler()
df_train_num_scaled = pd.DataFrame(scaler.fit_transform(df_train_num), columns=df_train_num.columns)
df_train = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

### Test

In [7]:
df_test_num = df_test.select_dtypes(include="number").drop('actor', axis=1)
df_test_cat = df_test.select_dtypes(include=['object'])

scaler = MinMaxScaler()
df_test_num_scaled = pd.DataFrame(scaler.fit_transform(df_test_num), columns=df_test_num.columns)
df_test = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [8]:
def label_encoder(array, label):
    new_array = np.zeros(len(array))
    for i in range(len(array)):
        if array[i] == label:
            new_array[i] = 1
    return new_array

## Vocal Channel

In [9]:
df_train_model_vc = df_train.copy()

In [10]:
df_test_model_vc = df_test.copy()

In [11]:
X_train_vc = df_train_model_vc.drop("vocal_channel", axis=1)
X_train_vc = np.array(pd.get_dummies(X_train_vc))
y_train_vc = label_encoder(df_train_model_vc["vocal_channel"], 'speech')

In [12]:
X_test_vc = df_test_model_vc.drop("vocal_channel", axis=1)
X_test_vc = np.array(pd.get_dummies(X_test_vc))
y_test_vc = label_encoder(df_test_model_vc["vocal_channel"], 'speech')

In [13]:
def logistic_regression(X_train_array, y_train_array):
    
    
    param_grid = {"penalty":["l2"],
              "C":[1e-4, 1e-3, 1e-2, 1e-1, 1e0,  1e2, 1e3], 
              "class_weight": ["balanced"],
              "solver": ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"],
              "max_iter":[800]}


    logreg=LogisticRegression()


    logreg_cv=GridSearchCV(logreg,
                      param_grid,
                      cv=5,
                      verbose=1,
                      n_jobs=-1,
                      error_score='raise'
                      )


    logreg_cv.fit(X_train_array,y_train_array)

    return logreg_cv

In [14]:
logreg_vocal_channel = logistic_regression(X_train_vc, y_train_vc)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


In [15]:
print("Tuned Logistic Regression Parameters: {}".format(logreg_vocal_channel.best_params_))

Tuned Logistic Regression Parameters: {'C': 1.0, 'class_weight': 'balanced', 'max_iter': 800, 'penalty': 'l2', 'solver': 'lbfgs'}


In [16]:
model_vocal_channel = LogisticRegression(C = 1.0, 
                                         class_weight = 'balanced', 
                                         max_iter = 800, 
                                         penalty = 'l2', 
                                         solver = 'lbfgs')
model_vocal_channel.fit(X_train_vc,y_train_vc)

LogisticRegression(class_weight='balanced', max_iter=800)

In [17]:
y_pred_logistic_vc = model_vocal_channel.predict(X_test_vc)

In [18]:
print(classification_report(y_test_vc,y_pred_logistic_vc))

              precision    recall  f1-score   support

         0.0       0.96      0.99      0.97       264
         1.0       0.99      0.97      0.98       360

    accuracy                           0.98       624
   macro avg       0.97      0.98      0.98       624
weighted avg       0.98      0.98      0.98       624



In [75]:
accuracy_vc = accuracy_score(y_test_vc,y_pred_logistic_vc)
f1_score_vc = f1_score(y_test_vc,y_pred_logistic_vc, average='weighted')

## Sex

In [19]:
df_train_model_s = df_train.copy()

In [20]:
df_test_model_s = df_test.copy()

In [21]:
X_train_s = df_train_model_s.drop("sex", axis=1)
X_train_s = np.array(pd.get_dummies(X_train_s))
y_train_s = label_encoder(df_train_model_s["sex"], 'M')

In [22]:
X_test_s = df_test_model_s.drop("sex", axis=1)
X_test_s = np.array(pd.get_dummies(X_test_s))
y_test_s = label_encoder(df_test_model_s["sex"], 'M')

In [23]:
logreg_sex = logistic_regression(X_train_s, y_train_s)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


In [24]:
print("Tuned Logistic Regression Parameters: {}".format(logreg_sex.best_params_))

Tuned Logistic Regression Parameters: {'C': 1.0, 'class_weight': 'balanced', 'max_iter': 800, 'penalty': 'l2', 'solver': 'lbfgs'}


In [25]:
model_sex = LogisticRegression(C = 1.0, 
                               class_weight = 'balanced', 
                               max_iter = 800, 
                               penalty = 'l2', 
                               solver = 'lbfgs')
model_sex.fit(X_train_s,y_train_s)

LogisticRegression(class_weight='balanced', max_iter=800)

In [26]:
y_pred_logistic_s = model_sex.predict(X_test_s)

In [27]:
print(classification_report(y_test_s,y_pred_logistic_s))

              precision    recall  f1-score   support

         0.0       1.00      0.71      0.83       312
         1.0       0.77      1.00      0.87       312

    accuracy                           0.85       624
   macro avg       0.88      0.85      0.85       624
weighted avg       0.88      0.85      0.85       624



In [73]:
accuracy_s = accuracy_score(y_test_s,y_pred_logistic_s)
f1_score_s = f1_score(y_test_s,y_pred_logistic_s, average='weighted')

## Emotional Intensity

In [28]:
df_train_model_ei = df_train.copy()

In [29]:
df_test_model_ei = df_test.copy()

In [30]:
X_train_ei = df_train_model_ei.drop("emotional_intensity", axis=1)
X_train_ei = np.array(pd.get_dummies(X_train_ei))
y_train_ei = label_encoder(df_train_model_ei["emotional_intensity"], 'normal')

In [31]:
X_test_ei = df_test_model_ei.drop("emotional_intensity", axis=1)
X_test_ei = np.array(pd.get_dummies(X_test_ei))
y_test_ei = label_encoder(df_test_model_ei["emotional_intensity"], 'normal')

In [32]:
logreg_emotional_intensity = logistic_regression(X_train_ei, y_train_ei)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


In [33]:
print("Tuned Logistic Regression Parameters: {}".format(logreg_emotional_intensity.best_params_))

Tuned Logistic Regression Parameters: {'C': 1.0, 'class_weight': 'balanced', 'max_iter': 800, 'penalty': 'l2', 'solver': 'lbfgs'}


In [34]:
model_emotional_intensity = LogisticRegression(C = 1.0, 
                                               class_weight = 'balanced', 
                                               max_iter = 800, 
                                               penalty = 'l2', 
                                               solver = 'lbfgs')
model_emotional_intensity.fit(X_train_ei,y_train_ei)

LogisticRegression(class_weight='balanced', max_iter=800)

In [35]:
y_pred_logistic_ei = model_emotional_intensity.predict(X_test_ei)

In [36]:
print(classification_report(y_test_ei,y_pred_logistic_ei))

              precision    recall  f1-score   support

         0.0       0.76      0.75      0.75       288
         1.0       0.79      0.80      0.79       336

    accuracy                           0.77       624
   macro avg       0.77      0.77      0.77       624
weighted avg       0.77      0.77      0.77       624



In [72]:
accuracy_ei = accuracy_score(y_test_ei,y_pred_logistic_ei)
f1_score_ei = f1_score(y_test_ei,y_pred_logistic_ei, average='weighted')

## Statement

In [37]:
df_train_model_st = df_train.copy()

In [38]:
df_test_model_st = df_test.copy()

In [39]:
X_train_st = df_train_model_st.drop("statement", axis=1)
X_train_st = np.array(pd.get_dummies(X_train_st))
y_train_st = label_encoder(df_train_model_st["statement"], 'Kids are talking by the door')

In [40]:
X_test_st = df_test_model_st.drop("statement", axis=1)
X_test_st = np.array(pd.get_dummies(X_test_st))
y_test_st = label_encoder(df_test_model_st["statement"], 'Kids are talking by the door')

In [41]:
logreg_statement = logistic_regression(X_train_st, y_train_st)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


In [42]:
print("Tuned Logistic Regression Parameters: {}".format(logreg_statement.best_params_))

Tuned Logistic Regression Parameters: {'C': 1.0, 'class_weight': 'balanced', 'max_iter': 800, 'penalty': 'l2', 'solver': 'lbfgs'}


In [43]:
model_statement = LogisticRegression(C = 1.0, 
                                     class_weight = 'balanced',
                                     max_iter = 800,
                                     penalty = 'l2', 
                                     solver = 'lbfgs')
model_statement.fit(X_train_st,y_train_st)

LogisticRegression(class_weight='balanced', max_iter=800)

In [44]:
y_pred_logistic_st = model_statement.predict(X_test_st)

In [45]:
print(classification_report(y_test_st,y_pred_logistic_st))

              precision    recall  f1-score   support

         0.0       0.96      0.82      0.89       312
         1.0       0.84      0.97      0.90       312

    accuracy                           0.89       624
   macro avg       0.90      0.89      0.89       624
weighted avg       0.90      0.89      0.89       624



## Emotion

In [46]:
df_train_model_e = df_train.copy()

In [47]:
df_test_model_e = df_test.copy()

In [48]:
def label_encoder_e(array):
    new_array = np.zeros(len(array))
    for i in range(len(array)):
        if array[i] == "calm":
            new_array[i] = 1
        elif array[i] == "happy":
            new_array[i] = 2
        elif array[i] == "sad":
            new_array[i] = 3
        elif array[i] == "angry":
            new_array[i] = 4
        elif array[i] == "fearful":
            new_array[i] = 5
        elif array[i] == "disgust":
            new_array[i] = 6
        elif array[i] == "surprised":
            new_array[i] = 7
    return new_array

In [49]:
X_train_e = df_train_model_e.drop("emotion", axis=1)
X_train_e = np.array(pd.get_dummies(X_train_e))
y_train_e = label_encoder_e(df_train_model_e["emotion"])

In [50]:
X_test_e = df_test_model_e.drop("emotion", axis=1)
X_test_e = np.array(pd.get_dummies(X_test_e))
y_test_e = label_encoder_e(df_test_model_e["emotion"])

In [51]:
logreg_emotion = logistic_regression(X_train_e, y_train_e)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


In [52]:
print("Tuned Logistic Regression Parameters: {}".format(logreg_emotion.best_params_))

Tuned Logistic Regression Parameters: {'C': 1.0, 'class_weight': 'balanced', 'max_iter': 800, 'penalty': 'l2', 'solver': 'newton-cholesky'}


In [53]:
model_emotion = LogisticRegression(C = 1.0, 
                                   class_weight = 'balanced', 
                                   max_iter = 800, 
                                   penalty = 'l2', 
                                   solver = 'newton-cholesky')
model_emotion.fit(X_train_e,y_train_e)

LogisticRegression(class_weight='balanced', max_iter=800,
                   solver='newton-cholesky')

In [54]:
y_pred_logistic_e = model_emotion.predict(X_test_e)

In [55]:
print(classification_report(y_test_e,y_pred_logistic_e))

              precision    recall  f1-score   support

         0.0       0.41      0.71      0.52        48
         1.0       0.45      0.73      0.56        96
         2.0       0.53      0.10      0.17        96
         3.0       0.50      0.06      0.11        96
         4.0       0.60      0.78      0.68        96
         5.0       0.60      0.43      0.50        96
         6.0       0.42      0.73      0.53        48
         7.0       0.43      0.73      0.54        48

    accuracy                           0.49       624
   macro avg       0.49      0.53      0.45       624
weighted avg       0.51      0.49      0.43       624



In [71]:
accuracy_e = accuracy_score(y_test_e,y_pred_logistic_e)
f1_score_e = f1_score(y_test_e,y_pred_logistic_e, average='weighted')

## Some Emotions
###### Ho considerato solo happy e sad

In [56]:
df_train_model_es = df_train.copy()

In [57]:
df_test_model_es = df_test.copy()

In [58]:
df_train_subset = df_train_model_es[(df_train_model_es["emotion"] == 'happy')] 
df_train_subset1 = df_train_model_es[(df_train_model_es["emotion"] == 'sad')]
df_train_subset_e = pd.concat([df_train_subset, df_train_subset1])



In [59]:
label_encoder_es = preprocessing.LabelEncoder()
X_train_es = df_train_subset_e.drop("emotion", axis=1)
X_train_es = np.array(pd.get_dummies(X_train_es))
y_train_es = label_encoder_es.fit_transform(df_train_subset_e['emotion'])

In [60]:
df_test_subset = df_test_model_es[(df_test_model_es["emotion"] == 'happy')] 
df_test_subset1 = df_test_model_es[(df_test_model_es["emotion"] == 'sad')]
df_test_subset_e = pd.concat([df_test_subset, df_test_subset1])


In [61]:
X_test_es = df_test_subset_e.drop("emotion", axis=1)
X_test_es = np.array(pd.get_dummies(X_test_es))
y_test_es = label_encoder_es.fit_transform(df_test_subset_e["emotion"])

In [62]:
logreg_emotion_subset = logistic_regression(X_train_es, y_train_es)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


In [63]:
print("Tuned Logistic Regression Parameters: {}".format(logreg_emotion_subset.best_params_))

Tuned Logistic Regression Parameters: {'C': 100.0, 'class_weight': 'balanced', 'max_iter': 800, 'penalty': 'l2', 'solver': 'liblinear'}


In [64]:
model_emotion_subset = LogisticRegression(C = 100.0, 
                                   class_weight = 'balanced', 
                                   max_iter = 800, 
                                   penalty = 'l2', 
                                   solver = 'liblinear')
model_emotion_subset.fit(X_train_es,y_train_es)

LogisticRegression(C=100.0, class_weight='balanced', max_iter=800,
                   solver='liblinear')

In [65]:
y_pred_logistic_es = model_emotion_subset.predict(X_test_es)

In [66]:
print(classification_report(y_test_es,y_pred_logistic_es))

              precision    recall  f1-score   support

           0       0.77      0.85      0.81        96
           1       0.84      0.74      0.78        96

    accuracy                           0.80       192
   macro avg       0.80      0.80      0.80       192
weighted avg       0.80      0.80      0.80       192



## Metriche per Logistic

In [76]:
# vocal_channel

vocal_channel_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
vocal_channel_metrics_b.loc["Accuracy"] = [None, None, None, None]
vocal_channel_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
vocal_channel_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [77]:
vocal_channel_metrics_b.loc["Accuracy"][0] = accuracy_vc
vocal_channel_metrics_b.loc["F1-Score (Weighted-Average)"][0] = f1_score_vc

In [88]:
vocal_channel_metrics_b.to_csv('vocal_channel_metric_logistic.csv', index=False)

In [138]:
y_score = model_vocal_channel.predict_proba(X_test_vc)
fpr, tpr, _ = roc_curve(y_test_vc, y_score[:, 1],pos_label=1)
roc_auc = auc(fpr, tpr)

In [139]:
vocal_channel_roc_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["fpr", "tpr", "auc"])
vocal_channel_roc_b.loc["fpr"] = [None, None, None, None]
vocal_channel_roc_b.loc["tpr"] = [None, None, None, None]
vocal_channel_roc_b.loc["auc"] = [None, None, None, None]
vocal_channel_roc_b

,LG,RF,DTB,AB
fpr,None,None,None,None
tpr,None,None,None,None
auc,None,None,None,None


In [140]:
vocal_channel_roc_b.loc["fpr"][0] = fpr
vocal_channel_roc_b.loc["tpr"][0] = tpr
vocal_channel_roc_b.loc["auc"][0] = roc_auc

In [141]:
vocal_channel_roc_b.to_csv('vocal_channel_scores_logistic.csv', index = False)

In [106]:
# sex

sex_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
sex_metrics_b.loc["Accuracy"] = [None, None, None, None]
sex_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
sex_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [107]:
sex_metrics_b.loc["Accuracy"][0] = accuracy_s
sex_metrics_b.loc["F1-Score (Weighted-Average)"][0] = f1_score_s

In [108]:
sex_metrics_b.to_csv('sex_metric_logistic.csv', index = False)

In [142]:
y_score_s = model_sex.predict_proba(X_test_s)
fpr_s, tpr_s, _ = roc_curve(y_test_s, y_score_s[:, 1],pos_label=1)
roc_auc_s = auc(fpr_s, tpr_s)

In [143]:
sex_roc_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["fpr", "tpr", "auc"])
sex_roc_b.loc["fpr"] = [None, None, None, None]
sex_roc_b.loc["tpr"] = [None, None, None, None]
sex_roc_b.loc["auc"] = [None, None, None, None]
sex_roc_b

,LG,RF,DTB,AB
fpr,None,None,None,None
tpr,None,None,None,None
auc,None,None,None,None


In [144]:
sex_roc_b.loc["fpr"][0] = fpr_s
sex_roc_b.loc["tpr"][0] = tpr_s
sex_roc_b.loc["auc"][0] = roc_auc_s

In [145]:
sex_roc_b.to_csv('sex_scores_logistic.csv', index = False)

In [120]:
# emotional_intensity

emotional_intensity_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
emotional_intensity_metrics_b.loc["Accuracy"] = [None, None, None, None]
emotional_intensity_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
emotional_intensity_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [121]:
emotional_intensity_metrics_b.loc["Accuracy"][0] = accuracy_ei
emotional_intensity_metrics_b.loc["F1-Score (Weighted-Average)"][0] = f1_score_ei

In [124]:
emotional_intensity_metrics_b.to_csv('emotional_intensity_metrics_logistic.csv', index=False)

In [146]:
y_score_ei = model_emotional_intensity.predict_proba(X_test_ei)
fpr_ei, tpr_ei, _ = roc_curve(y_test_ei, y_score_ei[:, 1], pos_label=1)
roc_auc_ei = auc(fpr_ei, tpr_ei)

In [147]:
# emotional_intensity

emotional_intensity_roc_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["fpr", "tpr", "auc"])
emotional_intensity_roc_b.loc["fpr"] = [None, None, None, None]
emotional_intensity_roc_b.loc["tpr"] = [None, None, None, None]
emotional_intensity_roc_b.loc["auc"] = [None, None, None, None]
emotional_intensity_roc_b

,LG,RF,DTB,AB
fpr,None,None,None,None
tpr,None,None,None,None
auc,None,None,None,None


In [148]:
emotional_intensity_roc_b.loc["fpr"][0] = fpr_ei
emotional_intensity_roc_b.loc["tpr"][0] = tpr_ei
emotional_intensity_roc_b.loc["auc"][0] = roc_auc_ei

In [149]:
emotional_intensity_roc_b.to_csv('emotional_intensity_scores_logistic.csv', index=False)

In [130]:
# emotion

emotion_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
emotion_metrics_b.loc["Accuracy"] = [None, None, None, None]
emotion_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
emotion_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [131]:
emotion_metrics_b.loc["Accuracy"][0] = accuracy_e
emotion_metrics_b.loc["F1-Score (Weighted-Average)"][0] = f1_score_e

In [133]:
emotion_metrics_b.to_csv('emotion_metrics_logistic.csv', index=False)